# Proyecto: Predicting Fraud in Financial Payment Services

# Proyecto para predecir el fraude financiero en los sistemas de pagos

Este proyecto es desarrollado por: Luis Daniel Trujillo, Jair Castro y Diego Ramirez

Fue construido en Python versión 3.8 utilizando Jupyter Book como interprete de Python y Visual Studio Code como editor de codigo.

In [2]:
# Cargue de las librerias

import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
#from xgboost.sklearn import XGBClassifier
#from xgboost import plot_importance, to_graphviz

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# Import de la base de datos
df = pd.read_csv("df.csv")

In [6]:
# Renombrando a las variables
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
print(df.head())

   step      type    amount     nameOrig  oldBalanceOrig  newBalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815        170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295         21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145           181.0            0.00   
3     1  CASH_OUT    181.00   C840083671           181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldBalanceDest  newBalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


Se crea un reporte para la previsualización y conocimiento de la base de datos

In [6]:

profile = ProfileReport(df, title='Análisis de las transacciones ', html={'style':{'full_width':True}})
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:4812: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
c:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:4814: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(var) / np.sqrt(2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print('\n Los tipos de transacciones fraudulentas son {}'.format(\
list(df.loc[df.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' 
                                                             # & 'TRANSFER'

dfFraudTransfer = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
dfFraudCashout = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]

print ('\n El número de transferencias fraudulentas es = {}'.\
       format(len(dfFraudTransfer))) # 4097

print ('\n El número de retiros fraudulentos es CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116

## 3. Limpieza de los datos
Del Analisis Exploratorio de Datos (AED)nos dimos cuenta que el fraude ocurre solo en las transferencias y los retiros. Por lo tanto vamos a trabajar con una base que tenga solo ese tipo de transacciones

In [9]:
# Se crea un sub data_frame con solo las transacciones TRANSFER y CASH_OUT
X = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]

randomState = 5
np.random.seed(randomState)

#X = X.loc[np.random.choice(X.index, 100000, replace = False)]

# Se pasa la columna fraude a un objeto llamado Y
Y = X['isFraud']
del X['isFraud']

# Se elimina las columnas irrelevantes para el AED
X = X.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)


# Se transforma el tipo de transacción en codificación binaria
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int) # convert dtype('O') to dtype(int)

### 3.1. Imputación de valores faltantes latentes
La base de datos tiene varias transacciones con balances en 0 en las cuentas receptoras, tanto en el momentos antes y después de una transacción con montos distintos a 0. La proporción de tales transacciones, es mucho más grande en las que son fraudulentas (50%) que en las que son genuinas (0.06%)


In [10]:
# Se separa las observaciones fraudulentas de las que no
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]

print('\nLa proporción de transacciones FRAUDULENTAS con \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 aun cuando el monto transado \'amount\' sea distinto de 0 es: {}'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceDest == 0) & \
(Xfraud.newBalanceDest == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud))))

print('\nLa proporción de transacciones GENUINAS con \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 aun cuando el monto transado \'amount\' sea distinto de 0 es: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceDest == 0) & \
(XnonFraud.newBalanceDest == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud))))


La proporción de transacciones FRAUDULENTAS con 'oldBalanceDest' = 'newBalanceDest' = 0 aun cuando el monto transado 'amount' sea distinto de 0 es: 0.4955558261293072

La proporción de transacciones GENUINAS con 'oldBalanceDest' = 'newBalanceDest' = 0 aun cuando el monto transado 'amount' sea distinto de 0 es: 0.0006176245277308345


Se hace la misma evaluación para las cuentas de origen

In [11]:
# Se separa las observaciones fraudulentas de las que no
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]

print('\nLa proporción de transacciones FRAUDULENTAS con \'oldBalanceOrig\' = \
\'newBalanceOrig\' = 0 aun cuando el monto transado \'amount\' sea distinto de 0 es: {}'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceOrig == 0) & \
(Xfraud.newBalanceOrig == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud))))

print('\nLa proporción de transacciones GENUINAS con \'oldBalanceOrig\' = \
\'newBalanceOrig\' = 0 aun cuando el monto transado \'amount\' sea distinto de 0 es: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceOrig == 0) & \
(XnonFraud.newBalanceOrig == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud))))


La proporción de transacciones FRAUDULENTAS con 'oldBalanceOrig' = 'newBalanceOrig' = 0 aun cuando el monto transado 'amount' sea distinto de 0 es: 0.0030439547059539756

La proporción de transacciones GENUINAS con 'oldBalanceOrig' = 'newBalanceOrig' = 0 aun cuando el monto transado 'amount' sea distinto de 0 es: 0.4737321319703598


Dado que el balance en 0 de las cuentas receptoras es un fuerte indicador de fraude, se procede a no hacer imputación del balance de la cuenta (en el momento antes de la transacción) con una distribución con una subsecuente ajuste para los montos transados. Si se hace esto, se maquillaría este indicador de fraude y haría que las transsacciones fraudulentas aparecieran como genuinas. Por lo tanto, se reemplazará el valor de 0 con -1, lo que será más util para la contrucción de un algoritmo de Machine Learning que detecte el fraude.

In [12]:
X.loc[(X.oldBalanceDest == 0) & (X.newBalanceDest == 0) & (X.amount != 0), \
      ['oldBalanceDest', 'newBalanceDest']] = - 1

Así como se demostró, los datos también tienen varias transacciones con balances en 0 en las cuentas de origen, antes y después de una transacción con montos distintos de 0. En este caso la proporción de tales transacciones es mucho menor en los casos de fraude (0.3%) comparado a las transacciones genuinas (47%). De manera similar al razonamiento anterior, en vez de imputar un valor numérico, se reemplaza los 0 con un valor nulo.

In [13]:
X.loc[(X.oldBalanceOrig == 0) & (X.newBalanceOrig == 0) & (X.amount != 0), \
      ['oldBalanceOrig', 'newBalanceOrig']] = np.nan

## 4. Ingeniería de variables
En vista de la posibilidad de que las cuentas con balance 0 sirvan para diferenciar entre las transacciones fraudulentas de las que no,  se tomó el proceso de imputación de la sección 3.1 un paso más allá y se creó dos nuevas columnas que registren el error en términos de monto en las cuentas de origen y receptoras para cada transacción. Estas nuevas variables resultaron ser importantes para obtener el mejor desempeño del algoritmo de ML que se usará al final.

In [14]:
X['errorBalanceOrig'] = X.newBalanceOrig + X.amount - X.oldBalanceOrig
X['errorBalanceDest'] = X.oldBalanceDest + X.amount - X.newBalanceDest

## 5. Visualización de datos
La mejor forma de confirmar que los datos contienen suficiente información para que el algoritmo de ML haga predicciones robustas, es intentar visualizar directamente la diferencia entre las transacciones fraudulentas de las genuinas. Bajo este principio, se visualizarán estas diferencias en los gráficos siguientes.

In [17]:
# Largo de la base
limit = len(X)

# Creación de la función plotStrip
def plotStrip(x, y, hue, figsize = (14, 9)):
    
    fig = plt.figure(figsize = figsize)
    colours = plt.cm.tab10(np.linspace(0, 1, 9))
    with sns.axes_style('ticks'):
        ax = sns.stripplot(x, y, \
             hue = hue, jitter = 0.4, marker = '.', \
             size = 4, palette = colours)
        ax.set_xlabel('')
        ax.set_xticklabels(['genuine', 'fraudulent'], size = 16)
        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(2)

        handles, labels = ax.get_legend_handles_labels()
        plt.legend(handles, ['Transfer', 'Cash out'], bbox_to_anchor=(1, 1), \
               loc=2, borderaxespad=0, fontsize = 16);
    return ax

### 5.1 Dispersión de las transacciones en el tiempo

Este gráfico muestra como las transacciones fraudulentas y las genuinas tienen distintas connotaciones cuando su dispersión es vista en el tiempo. Está claro que las transacciones fraudulentas están más homogeneamente distribuidas en el tiempo en comparación a las genuinas. También es destacable que los retiros superan en número a las transferencias dentro de las transacciones genuinas, en contraste a la distribución balanceada que hay en las transacciones con fraude. Nótese además que se usó el parámetro _jitter_ en la función Plotstrip que se diseñó, para poder separar y diferenciar las transacciones que ocurrian al mismo tiempo.

In [18]:
ax = plotStrip(Y[:limit], X.step[:limit], X.type[:limit])
ax.set_ylabel('time [hour]', size = 16)
ax.set_title('Striped vs. homogenous fingerprints of genuine and fraudulent \
transactions over time', size = 20);

C:\Users\luist\AppData\Local\Temp\ipykernel_7328\1949919385.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  ax = plotStrip(Y[:limit], X.step[:limit], X.type[:limit])


TypeError: stripplot() takes from 0 to 1 positional arguments but 2 positional arguments (and 4 keyword-only arguments) were given

<Figure size 1400x900 with 0 Axes>

### 5.2 Dispersión de las transacciones en los montos
Los 2 gráficos muestran que aunque  la presencia de fraude en una transacción puede ser discernida por el monto de la transacción original, la variable creada de error en el balance es más efectiva en hacer esta distinción.

In [ ]:
limit = len(X)
ax = plotStrip(Y[:limit], X.amount[:limit], X.type[:limit], figsize = (14, 9))
ax.set_ylabel('amount', size = 16)
ax.set_title('Same-signed fingerprints of genuine \
and fraudulent transactions over amount', size = 18);

### 5.3. Dispersión de los errores en el balance en las cuentas receptoras

In [ ]:
limit = len(X)
ax = plotStrip(Y[:limit], - X.errorBalanceDest[:limit], X.type[:limit], \
              figsize = (14, 9))
ax.set_ylabel('- errorBalanceDest', size = 16)
ax.set_title('Opposite polarity fingerprints over the error in \
destination account balances', size = 18);

### 5.4. Separación de las transacciones genuinas y fraudulentas